In [1]:
import OpenDartReader
from bs4 import BeautifulSoup
import pandas as pd
import time
from html_table_parser import parser_functions as parser
from tqdm import tqdm
from selenium import webdriver
from datetime import datetime,timedelta


api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
# today = datetime.today().strftime("%Y%m%d") #오늘
today = (datetime.today()-timedelta(days=1)).strftime("%Y%m%d") #오늘

dart_list = dart.list(start=today,end=today)
lists = dart_list[dart_list['report_nm'].str.contains('시설투자')&~dart_list['report_nm'].str.contains('자회사')]
print(f'{today}_{len(lists)}건')
# 기재 정정이 아니면 아래와 같이
def invest_equipment(rcp_no,corp):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table') #표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    
    tt = tt.T
    # # print(tt)
    if '투자대상' in tt[1].iloc[0]:
        tt = tt[[0,1,2,3,4,6,7,8,9]]
        tt.columns = ['투자구분','투자대상','투자금액(원)','자기자본(원)','자기자본대비(%)','투자목적','시작일','종료일','결정일']
        tt = tt.drop(index=[0,1,3])
        tt.insert(0,'기업',corp)
    else:
        tt = tt[[0,1,2,3,5,6,7,8]]
        tt.columns = ['투자구분','투자금액(원)','자기자본(원)','자기자본대비(%)','투자목적','시작일','종료일','결정일']
        if tt['투자목적'].iloc[2] == None:
            
            tt['투자목적'] = tt['투자목적'].shift(1)
            tt['결정일'] = tt['결정일'].shift(1)
        
        if len(tt.index) == 4:
            tt = tt.drop(index=[0,1,3])
        elif len(tt.index) == 3:
            tt = tt.drop(index=[0,1])
          
        tt.insert(1,'투자대상','-')
        tt.insert(0,'기업',corp)
    tt.insert(0,'공시구분','최초공시')
    tt.insert(0,'조회날짜',today)
    time.sleep(1)

    return tt

# 기재 정정이면 아래와 같이
def invest_equipment_adj(rcp_no,corp):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table') #표를 불러옴
    p=parser.make2d(tables[-1])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt.T
    
    if '투자대상' in tt[1].iloc[0]:
        tt = tt[[0,1,2,3,4,6,7,8,9]]
        tt.columns = ['투자구분','투자대상','투자금액(원)','자기자본(원)','자기자본대비(%)','투자목적','시작일','종료일','결정일']
        tt = tt.drop(index=[0,1,3])
        tt.insert(0,'기업',corp)
    else:
        tt = tt[[0,1,2,3,5,6,7,8]]
        tt.columns = ['투자구분','투자금액(원)','자기자본(원)','자기자본대비(%)','투자목적','시작일','종료일','결정일']
        if tt['투자목적'].iloc[-1] == None:
            tt['투자목적'] = tt['투자목적'].shift(1)
            tt['결정일'] = tt['결정일'].shift(1)
        
        if len(tt.index) == 4:
            tt = tt.drop(index=[0,1,3])
        elif len(tt.index) == 3:
            tt = tt.drop(index=[0,1])
        
            
        tt.insert(1,'투자대상','-')
        tt.insert(0,'기업',corp)
    tt.insert(0,'공시구분','기재정정')
    tt.insert(0,'조회날짜',today)
    time.sleep(1)

    return tt

df = pd.DataFrame()
for i,re_nm in lists.iterrows():
    try:
        if '기재정정' in re_nm['report_nm']:
            rcp_no = re_nm['rcept_no']
            corp = re_nm['corp_name']
            equipment_df = invest_equipment_adj(rcp_no,corp) # 기재정정
            print(f'{corp}_{len(lists)}건')
            df = pd.concat([df,equipment_df])
        else:
            rcp_no = re_nm['rcept_no']
            corp = re_nm['corp_name']   
            equipment_df = invest_equipment(rcp_no,corp) # 최초공시
            print(f'{corp}_{len(lists)}건')
            df = pd.concat([df,equipment_df])
    except Exception as e:
        print(re_nm['corp_name'],e)

file_name = f'C:\투자기록\정리\코딩\장내매수\시설투자_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
df.to_excel(
    writer, 
    sheet_name='Sheet3',
    startcol = 0,
    startrow = writer.sheets['Sheet3'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()


20241008_3건
한빛레이저_3건
진바이오텍_3건
탑머티리얼_3건
